In [1]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot

En primera base se probó con la señal del par EUR-USD para ver esta como se comportaba, ahora probaremos con varias señales de entrada, para ver como se comporta la señal EUR-USD.

Para empezar se probará con los pares EUR/USD y GBP/USD para predecir la señal EUR/USD

In [17]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("../data/TrueFX/GBP-USD/GBPUSD-preprocesado-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [18]:
data_GBPvsUSD.head()

,bid,ask
date,,
2018-01-02 00:00:40,1.34979,1.35176
2018-01-02 00:01:00,1.34918,1.35208
2018-01-02 00:01:10,1.34915,1.35354
2018-01-02 00:01:20,1.34915,1.35386
2018-01-02 00:01:30,1.34938,1.35440


In [19]:
data_EURvsUSD.head()

,bid,ask
date,,
2018-01-01 23:58:00,1.20093,1.20115
2018-01-01 23:58:10,1.20093,1.20111
2018-01-01 23:58:20,1.20082,1.20111
2018-01-01 23:58:30,1.20043,1.20110
2018-01-01 23:58:50,1.20011,1.20113


In [20]:
print('tamaño de la señal EUR/USD', data_EURvsUSD.shape)
print('tamaño de la señal GBP/USD', data_GBPvsUSD.shape)

tamaño de la señal EUR/USD (183535, 2)
tamaño de la señal GBP/USD (185443, 2)


como se puede ver, las dos señales no tienen el mismo tamaño, como la señal predominante es el par EUR/USD, el indice de este será el tenido en cuenta para la construcción de la multiseñal.

procedemos a cargar la clase "EUtilities" hecha con anterioridad para tratar estos datos...

In [21]:
%run ../code/EUtilities.py
EU = EUtilities()

para poder utiliar la función "multi_signal" se deben cumplir ciertas caracteristicas, entre ellas, que los dataframe a utilizar tenga su columna llamada "bid"...

In [24]:
#se construye el dataset EUR/USD con las dos señales en el, para su posterior uso
data_EURvsUSD = EU.multi_signal(data_EURvsUSD,data_GBPvsUSD)

In [27]:
data_EURvsUSD[:10]

,bid,ask,bidnew
date,,,
2018-01-01 23:58:00,1.20093,1.20115,1.20093
2018-01-01 23:58:10,1.20093,1.20111,1.20093
2018-01-01 23:58:20,1.20082,1.20111,1.20082
2018-01-01 23:58:30,1.20043,1.20110,1.20043
2018-01-01 23:58:50,1.20011,1.20113,1.20011
2018-01-01 23:59:00,1.20015,1.20110,1.20015
2018-01-01 23:59:10,1.20028,1.20110,1.20028
2018-01-01 23:59:50,1.20015,1.20110,1.20015
2018-01-02 00:00:40,1.20015,1.20105,1.34979


ya con la multiseñal se procede a crear el X...

In [28]:
#los datos del par EUR/USD ""
X_EURUSD, y_EURUSD, bt_EURUSD = EU.build_dataset(data_EURvsUSD.iloc[:,0],window=3,binary_target=True,PNL=True
                                             ,delete_constant_values=True)
#los datos del par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD,_ = EU.build_dataset(data_EURvsUSD.iloc[:,2],window=3,binary_target=False,delete_constant_values=True)

In [29]:
X_EURUSD.shape, X_GBPUSD.shape

((156578, 4), (161098, 3))

debido a que son diferentes señales, al habilitar la opción de <i>"delete_constant_values"</i> en la función <i>"build_dataset"</i> esto hace que las señales resultantes no sean iguales en tamaño, por ende para entrenar se cogeran los valores de los indices de la señal a medir, si este no está en la otra señal, se coloca lo de la señal EUR/USD

In [30]:
X_GBPUSD = X_GBPUSD.loc[X_EURUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((156578, 4), (156578, 3))

In [31]:
X_GBPUSD[:10]

,0,1,2
date,,,
2018-01-01 23:58:50,1.20093,1.20093,1.20082
2018-01-01 23:59:00,1.20093,1.20082,1.20043
2018-01-01 23:59:10,1.20082,1.20043,1.20011
2018-01-01 23:59:50,1.20043,1.20011,1.20015
2018-01-02 00:00:40,1.20011,1.20015,1.20028
2018-01-02 00:01:00,1.20028,1.20015,1.34979
2018-01-02 00:03:00,1.20030,1.34918,1.34915
2018-01-02 00:03:30,1.34915,1.34940,1.20026
2018-01-02 00:03:40,NaN,NaN,NaN


In [32]:
#para cambiar los valores NaN por los que hay en la señal EUR/USD
X_GBPUSD = X_GBPUSD.fillna(X_EURUSD,axis=0)

se unen para entrenar la multiseñal...

In [33]:
X_EURUSD.insert(loc=3,column=3,value=X_GBPUSD.iloc[:,0])
X_EURUSD.insert(loc=4,column=4,value=X_GBPUSD.iloc[:,1])
X_EURUSD.insert(loc=5,column=5,value=X_GBPUSD.iloc[:,2])

In [34]:
X_EURUSD.head()

,0,1,2,3,4,5,PNL
date,,,,,,,
2018-01-01 23:58:50,1.20093,1.20093,1.20082,1.20093,1.20093,1.20082,-0.00039
2018-01-01 23:59:00,1.20093,1.20082,1.20043,1.20093,1.20082,1.20043,-0.00032
2018-01-01 23:59:10,1.20082,1.20043,1.20011,1.20082,1.20043,1.20011,0.00004
2018-01-01 23:59:50,1.20043,1.20011,1.20015,1.20043,1.20011,1.20015,0.00013
2018-01-02 00:00:40,1.20011,1.20015,1.20028,1.20011,1.20015,1.20028,-0.00013
